In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
# --------------------------------------------------------------------------
# | БЛОК 1: Настройка файлов и весов                                       |
# --------------------------------------------------------------------------

# Укажите пути к вашим файлам с предсказаниями
file_1_path = 'C:/Users/Николай/PycharmProjects/FlightRank_2025/submissions/submission_34.csv'
file_2_path = 'C:/Users/Николай/PycharmProjects/FlightRank_2025/submissions/submission_37.csv'
file_3_path = 'C:/Users/Николай/PycharmProjects/FlightRank_2025/submissions/submission_40.csv'
file_4_path = 'C:/Users/Николай/PycharmProjects/FlightRank_2025/submissions/submission_36.csv'

# Укажите скоры (веса) ваших моделей.
# Модели с более высоким скором будут иметь большее влияние.
weight_1 = 0.50004
weight_2 = 0.49912
weight_3 = 0.49664
weight_4 = 0.49793

# Путь для сохранения итогового файла
output_path = 'C:/Users/Николай/PycharmProjects/FlightRank_2025/submissions/submission_42.csv'

print("--- Настройки ---")
print(f"Файл 1: {file_1_path} (вес: {weight_1})")
print(f"Файл 2: {file_2_path} (вес: {weight_2})")
print(f"Файл 3: {file_3_path} (вес: {weight_3})")
print(f"Файл 4: {file_4_path} (вес: {weight_4})")
print(f"Итоговый файл: {output_path}\n")

--- Настройки ---
Файл 1: C:/Users/Николай/PycharmProjects/FlightRank_2025/submissions/submission_34.csv (вес: 0.50004)
Файл 2: C:/Users/Николай/PycharmProjects/FlightRank_2025/submissions/submission_37.csv (вес: 0.49912)
Файл 3: C:/Users/Николай/PycharmProjects/FlightRank_2025/submissions/submission_40.csv (вес: 0.49664)
Файл 4: C:/Users/Николай/PycharmProjects/FlightRank_2025/submissions/submission_36.csv (вес: 0.49793)
Итоговый файл: C:/Users/Николай/PycharmProjects/FlightRank_2025/submissions/submission_42.csv


In [3]:
# --------------------------------------------------------------------------
# | БЛОК 2: Загрузка и подготовка данных                                   |
# --------------------------------------------------------------------------

print("Шаг 1: Загрузка файлов с предсказаниями...")
df1 = pd.read_csv(file_1_path)
df2 = pd.read_csv(file_2_path)
df3 = pd.read_csv(file_3_path)
df4 = pd.read_csv(file_4_path)

Шаг 1: Загрузка файлов с предсказаниями...


In [4]:
# --------------------------------------------------------------------------
# | БЛОК 3: Преобразование рангов в оценки (Scores)                        |
# --------------------------------------------------------------------------

# Вспомогательная функция для преобразования рангов в оценки
def rank_to_score(series):
    n = len(series)
    return 1.0 - (series - 1) / n

print("Шаг 2: Преобразование рангов в оценки для каждой группы...")

df1['score_1'] = df1.groupby('ranker_id')['selected'].transform(rank_to_score)
df2['score_2'] = df2.groupby('ranker_id')['selected'].transform(rank_to_score)
df3['score_3'] = df3.groupby('ranker_id')['selected'].transform(rank_to_score)
df4['score_4'] = df4.groupby('ranker_id')['selected'].transform(rank_to_score)

print("Преобразование завершено.\n")

Шаг 2: Преобразование рангов в оценки для каждой группы...
Преобразование завершено.


In [5]:
# --------------------------------------------------------------------------
# | БЛОК 4: Объединение и взвешенное усреднение                             |
# --------------------------------------------------------------------------

print("Шаг 3: Объединение оценок и взвешенное усреднение...")

# Последовательно объединяем DataFrame'ы по 'Id'
merged_df = pd.merge(
    df1[['Id', 'ranker_id', 'score_1']],
    df2[['Id', 'score_2']],
    on='Id'
)
merged_df = pd.merge(
    merged_df,
    df3[['Id', 'score_3']],
    on='Id'
)

merged_df = pd.merge(
    merged_df,
    df4[['Id', 'score_4']],
    on='Id'
)

# Вычисляем взвешенную среднюю оценку для трех моделей
total_weight = weight_1 + weight_2 + weight_3 + weight_4 
merged_df['blended_score'] = (
    merged_df['score_1'] * weight_1 +
    merged_df['score_2'] * weight_2 +
    merged_df['score_3'] * weight_3 +
    merged_df['score_4'] * weight_4
) / total_weight

print("Усреднение завершено.\n")

Шаг 3: Объединение оценок и взвешенное усреднение...
Усреднение завершено.


In [6]:
# --------------------------------------------------------------------------
# | БЛОК 5: Преобразование итоговых оценок в ранги и сохранение              |
# --------------------------------------------------------------------------

print("Шаг 4: Преобразование итоговых оценок обратно в ранги...")

# Группируем по ranker_id и ранжируем по новой усредненной оценке
merged_df['selected'] = merged_df.groupby('ranker_id')['blended_score'].transform(
    lambda s: s.rank(method='first', ascending=False)
).astype(int)

# Создаем финальный DataFrame в нужном формате
submission_df = merged_df[['Id', 'ranker_id', 'selected']]

print("Ранжирование завершено.\n")

# Сохраняем результат
print(f"Шаг 5: Сохранение итогового файла в {output_path}...")
submission_df.to_csv(output_path, index=False)

print("\n--- Готово! ---")
print("Пример содержимого итогового файла:")
print(submission_df.head())

Шаг 4: Преобразование итоговых оценок обратно в ранги...
Ранжирование завершено.

Шаг 5: Сохранение итогового файла в C:/Users/Николай/PycharmProjects/FlightRank_2025/submissions/submission_42.csv...

--- Готово! ---
Пример содержимого итогового файла:
         Id                         ranker_id  selected
0  18144679  c9373e5f772e43d593dd6ad2fa90f67a         9
1  18144680  c9373e5f772e43d593dd6ad2fa90f67a        53
2  18144681  c9373e5f772e43d593dd6ad2fa90f67a       240
3  18144682  c9373e5f772e43d593dd6ad2fa90f67a        80
4  18144683  c9373e5f772e43d593dd6ad2fa90f67a        92
